# Predittore di precipitazione nella città di Udine (S. Osvaldo)

Questo programma aiuta a predirre la situazione di pioggia del giorno sucessivo partendo da alcuni dati del giorno attuale.

Nello specifico, utilizza:
* mm di Pioggia oggi
* Temperature (massima, minima, media)
* Umidità (massima, media)
* Anno, Mese e Giorno (come posizione nel mese)
* Vento (massimo, medio, direzione)
* Irradiamento solare (in J/m2)
* Pressione atmosferica media (in Pascal)

L'output invece sarà una stringa, che varierà tra:
* "Nulla o Pochissima" (0 - 0.4 mm / giorno)
* "Leggera" (0.4 - 2.9 mm / giorno)
* "Media" (3 - 7.9 mm / giorno)
* "Forte" (> 8 mm / giorno)

Per chi è più esperto, è possibile anche modificare il codice provando altri modelli con altri parametri per ottenere risultati migliori

Per iniziare, importeremo alcuni moduli necessari per il funzionamento dei modelli. Sucessivamente importeremo il file contente i dati relativi al passato meteorologico, che per comodità è già stato pulito

In [64]:
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from imblearn.over_sampling import SMOTE

df = pd.read_parquet("/content/file.parquet")

Ora andremo a definire le varie colonne del dataframe, distinguendole tra input e output, e tra numeriche e categoriche.
Andremo anche a creare una lista di colonne da ignorare durante la fase di training, che però in questo caso è vuota dato che useremo tutte le variabili

In [65]:
cols_to_ignore = []
cols_to_ignore.append("PioggiaDomani")

input_cols = df.columns.drop("PioggiaDomani")
output_cols = ["PioggiaDomani"]

numerical_cols = ["Anno", "TempMin [°C]", "TempMed [°C]", "TempMax [°C]", "UmiditaMed [%]", "UmiditaMax [%]", "VentoMed [km/h]", "VentoMax [km/h]", "Radiazione [J/m2]", "Pressione [Pa]", "mmPioggia"]
categorical_cols = ["Mese", "PosizioneMese", "DirVentoMax"]

Dopo aver definito le colonne, procederemo a trasformare tutti i dati:
* Andremo a riempire i valori vuoti di ogni colonna con SimpleImputer di scikit-learn
* Poi standardizzeremo i dati in modo da migliorare l'interpretabilità del modello e possibilmente anche la sua accuratezza generale
* Infine, trasformeremo le variabili categoriche (non numeriche) in un formato leggibile dai modelli tramite OneHotEncoding


In [66]:
imputer = sklearn.impute.SimpleImputer(strategy="mean")
imputer.fit(df[numerical_cols])
df[numerical_cols] = imputer.transform(df[numerical_cols])

scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(df[numerical_cols])
df[numerical_cols] = scaler.transform(df[numerical_cols])

encoder = sklearn.preprocessing.OneHotEncoder()
encoder.fit(df[categorical_cols])
encoded = encoder.transform(df[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

Una volta preparati i dati, li possiamo dividere in due set:
* Uno di training (da usare per addestrare il modello)
* E uno di test (per testare il modello e quanto affidabile è)

Inoltre, creeremo anche un secondo set di dati di training modificati tramite oversampling con SMOTE, per usarli su due modelli diversi, che poi approfondiremo

In [67]:
input_cols = df.columns.drop("PioggiaDomani")
output_cols = ["PioggiaDomani"]

df_train = df[df["Anno"] < 1] # <= 2023
df_test = df[df["Anno"] > 1] # >= 2024

train_input = df_train[input_cols]
train_output = df_train[output_cols]
test_input = df_test[input_cols]
test_output = df_test[output_cols]

sm = SMOTE(random_state=42)
train_input_resampled, train_output_resampled = sm.fit_resample(train_input, train_output)

Ora passiamo alla creazione dei due modelli:
* Saranno tutti e due basati sulla funzione logistica di LogisticRegression di sklearn
* Uno verrà addestrato con dei dati con proporzioni e probabilità reali
* L'altro sarà addestrato su dati misti ad altri creati sinteticamente per evitare che le minoranze vengano sottovalutate

In [68]:
model = sklearn.linear_model.LogisticRegression(penalty="l2", class_weight="balanced", solver="liblinear", max_iter=1000000, tol=1e-100)
model_resampled = sklearn.linear_model.LogisticRegression(penalty="l2", class_weight="balanced", solver="liblinear", max_iter=1000000, tol=1e-100)

model.fit(train_input, train_output.values.ravel())
model_resampled.fit(train_input_resampled, train_output_resampled.values.ravel())

LogisticRegression(class_weight='balanced', max_iter=1000000,
                   solver='liblinear', tol=1e-100)

Poi, li faremo predirre le precipitazioni del dataframe di test precedentemente creato

In [69]:
test_output_pred = model.predict(test_input)
test_output_pred_resampled = model_resampled.predict(test_input)

Questi risultati saranno quindi salvati ed usati per creare una matrice di confusione, che ci mostrerà dove i nostri modelli sono più accurati, e dove invece non lo sono

Valori più vicini ad 1 significheranno predizioni più accurate, mentre valori vicini a 0 significheranno predizioni quasi mai giuste


Per rendere il modello più flessibile, è stato aggiunto un parametro "weight_close_pred" che va da 0 ad 1, ed indica il margine di errore accettato nelle predizioni

Per esempio, un margine di 0.3 o 30%, significa che per il 30% delle volte, una predizione "vicina" è comunque considerata corretta
(per esempio, predirre "Leggera" quando la realtà è "Nulla" o "Media", risulterà nel 30% delle volte nell'essere presa come predizione valida)



In [70]:
conf_matx = sklearn.metrics.confusion_matrix(test_output, test_output_pred, normalize="true", labels=["Nulla o Pochissima", "Leggera", "Media", "Forte"])
conf_matx_resampled = sklearn.metrics.confusion_matrix(test_output, test_output_pred_resampled, normalize="true", labels=["Nulla o Pochissima", "Leggera", "Media", "Forte"])

weight_close_pred = 0.3

nulla_accuracy = conf_matx[0][0] + conf_matx[0][1]*weight_close_pred
leggera_accuracy = conf_matx[1][1] + conf_matx[1][0]*weight_close_pred + conf_matx[1][2]*weight_close_pred
media_accuracy = conf_matx[2][2] + conf_matx[2][1]*weight_close_pred + conf_matx[2][3]*weight_close_pred
forte_accuracy = conf_matx[3][3] + conf_matx[3][2]*weight_close_pred

nulla_accuracy_resampled = conf_matx_resampled[0][0] + conf_matx_resampled[0][1]*weight_close_pred
leggera_accuracy_resampled = conf_matx_resampled[1][1] + conf_matx_resampled[1][0]*weight_close_pred + conf_matx_resampled[1][2]*weight_close_pred
media_accuracy_resampled = conf_matx_resampled[2][2] + conf_matx_resampled[2][1]*weight_close_pred + conf_matx_resampled[2][3]*weight_close_pred
forte_accuracy_resampled = conf_matx_resampled[3][3] + conf_matx_resampled[3][2]*weight_close_pred

print(f'Margine di accettazione = {weight_close_pred*100}% (si contano anche il {weight_close_pred*100}% delle predizioni "vicine" come comunque corrette)\n')
print(f"### PERCENTUALI DI ACCURATEZZA STORICA PER OGNI MODELLO:\n")
print(f"# Modello trainato su dati standard:\n\n- Nulla o Pochissima ☀️ {round(nulla_accuracy*100, 2)}%\n- Leggera 🌦️ {round(leggera_accuracy*100, 2)}%\n- Media 🌧️ {round(media_accuracy*100, 2)}%\n- Forte ⛈️ {round(forte_accuracy*100, 2)}%")
print(f"\n# Modello trainato su dati resampled (SMOTE):\n\n- Nulla o Pochissima ☀️ {round(nulla_accuracy_resampled*100, 2)}%\n- Leggera 🌦️ {round(leggera_accuracy_resampled*100, 2)}%\n- Media 🌧️ {round(media_accuracy_resampled*100, 2)}%\n- Forte ⛈️ {round(forte_accuracy_resampled*100, 2)}%\n\n")

Margine di accettazione = 30.0% (si contano anche il 30.0% delle predizioni "vicine" come comunque corrette)

### PERCENTUALI DI ACCURATEZZA STORICA PER OGNI MODELLO:

# Modello trainato su dati standard:

- Nulla o Pochissima ☀️ 87.36%
- Leggera 🌦️ 30.62%
- Media 🌧️ 16.41%
- Forte ⛈️ 39.33%

# Modello trainato su dati resampled (SMOTE):

- Nulla o Pochissima ☀️ 59.84%
- Leggera 🌦️ 57.38%
- Media 🌧️ 47.44%
- Forte ⛈️ 29.33%




Possiamo notare come il modello base possiede una accuratezza storica (o meglio, recall, per usare un termine più corretto e tecnico) relativamente alta per i casi più estremi, con pioggia nulla e pioggia forte

Al contrario, il secondo modello con SMOTE ha performato leggermente meglio nei casi di mezzo (Leggera e Media), a discapito dei casi estremi


Possiamo quindi decidere di performare quello che in gergo viene definito un "ensemble". Uniamo quindi i due modelli, prendendo i vantaggi di entrambi.

In [71]:
print(f"\n# Modello combinato (prende la predizione più affidabile da ogni modello):\n\n- Nulla o Pochissima ☀️ {max(round(nulla_accuracy*100, 2), round(nulla_accuracy_resampled*100, 2))}%\n- Leggera 🌦️ {max(round(leggera_accuracy*100, 2), round(leggera_accuracy_resampled*100, 2))}%\n- Media 🌧️ {max(round(media_accuracy*100, 2), round(media_accuracy_resampled*100, 2))}%\n- Forte ⛈️ {max(round(forte_accuracy*100, 2), round(forte_accuracy_resampled*100, 2))}%\n\n")


# Modello combinato (prende la predizione più affidabile da ogni modello):

- Nulla o Pochissima ☀️ 87.36%
- Leggera 🌦️ 57.38%
- Media 🌧️ 47.44%
- Forte ⛈️ 39.33%




Una volta ideato il modello, possiamo crearlo in modo che, per qualsiasi input che gli passiamo, ci dica il caso di pioggia più probabile per il giorno dopo e la sua confidenza, aggiustata per l'accuratezza storica sopra calcolata

In [72]:
def final_model_predict(input_to_predict):
    df_input_to_predict = pd.DataFrame(input_to_predict, index=[1])

    numerical_cols = ["Anno", "TempMin [°C]", "TempMed [°C]", "TempMax [°C]", "UmiditaMed [%]", "UmiditaMax [%]", "VentoMed [km/h]", "VentoMax [km/h]", "Radiazione [J/m2]", "Pressione [Pa]", "mmPioggia"]
    categorical_cols = ["Mese", "PosizioneMese", "DirVentoMax"]

    df_input_to_predict[numerical_cols] = imputer.transform(df_input_to_predict[numerical_cols])

    df_input_to_predict[numerical_cols] = scaler.transform(df_input_to_predict[numerical_cols])

    encoded = encoder.transform(df_input_to_predict[categorical_cols]).toarray()
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(categorical_cols), index=df_input_to_predict.index)
    df_input_to_predict = pd.concat([df_input_to_predict.drop(columns=categorical_cols), encoded_df], axis=1)


    model_accuracy_from_output = {"Nulla o Pochissima": nulla_accuracy, "Leggera": leggera_accuracy, "Media": media_accuracy, "Forte": forte_accuracy}
    model_resampled_accuracy_from_output = {"Nulla o Pochissima": nulla_accuracy_resampled, "Leggera": leggera_accuracy_resampled, "Media": media_accuracy_resampled, "Forte": forte_accuracy_resampled}
    emoji_from_output = {"Nulla o Pochissima": "☀️", "Leggera": "🌦️", "Media": "🌧️", "Forte": "⛈️"}

    output = model.predict(df_input_to_predict)[0]
    output_resampled = model_resampled.predict(df_input_to_predict)[0]

    idx_output = list(model.classes_).index(output)
    idx_output_resampled = list(model_resampled.classes_).index(output_resampled)

    output_conf = round(model.predict_proba(df_input_to_predict)[0][idx_output]*100, 2)
    output_resampled_conf = round(model_resampled.predict_proba(df_input_to_predict)[0][idx_output_resampled]*100, 2)

    accuracy_model = round(model_accuracy_from_output[output]*100, 2)
    accuracy_model_resampled = round(model_resampled_accuracy_from_output[output_resampled]*100, 2)

    if accuracy_model >= accuracy_model_resampled:
        chosen_output = output
        chosen_model = model
        emoji = emoji_from_output[output]
        print(f"\nDomani sarà --> {output} {emoji} con {round(output_conf*accuracy_model/100, 2)}% di confidenza contando l'accuratezza storica\n")
    else:
        chosen_output = output_resampled
        chosen_model = model_resampled
        emoji = emoji_from_output[output_resampled]
        print(f"\nDomani sarà --> {output_resampled} {emoji} con {round(output_resampled_conf*accuracy_model_resampled/100, 2)}% di confidenza contando l'accuratezza storica\n")

Ora possiamo immettere i dati della giornata in questione nel codice sotto

In [73]:
input_to_predict = {
    "Anno": 2025, # Anno della predizione
    "Mese": "Lug", # Mese della predizione (abbreviato, 3 lettere, prima lettera maiuscola)
    "PosizioneMese": "Fine", # In che fase del mese si trova ("Inizio", "Meta", "Fine")
    "DirVentoMax": "SO", # Direzione cardinale del vento massima velocità (Anche combinazioni es. "SO")
    "TempMin [°C]": 16.3, # Temperatura minima registrata (°C)
    "TempMed [°C]": 22.7, # Temperatura media registrata (°C)
    "TempMax [°C]": 28.6, # Temperatura massima registrata (°C)
    "UmiditaMed [%]": 64, # Umidità media registrata (%)
    "UmiditaMax [%]": 91, # Umidità massima registrata (%)
    "VentoMed [km/h]": 6, # Velocità media del vento (km/h)
    "VentoMax [km/h]": 23, # Velocità massima del vento (km/h)
    "Radiazione [J/m2]": 27270000, # Irradiamento solare totale [J/m2]
    "Pressione [Pa]": 100430, # Pressiona atmosferica media (Pascal)
    "mmPioggia": 0 # Millimetri di pioggia totali (anche scritti come Litri per m2)
}

Ed ora possiamo chiamare la funzione con i dati appena inseriti

In [74]:
final_model_predict(input_to_predict)


Domani sarà --> Nulla o Pochissima ☀️ con 43.68% di confidenza contando l'accuratezza storica



Se si vuole, si possono anche modificare i dati a proprio piacimento, modificando i valori tra i due punti (:) e la virgola (,) seguendo le unità di misura quando questi sono di tipo numerico.

Quando invece sono di tipo testuale (stringa), ricordarsi di racchiudere sempre il testo tra le virgolette (" ")

# Conclusione

Questo modello non è completo, e non sarà nemmeno il più accurato o efficiente, tuttavia offre un buon punto di partenza per modellazioni meteorologiche su dataset locali.

Spero possa servire utile, e spero abbia servito come prova del fatto che tramite programmi del genere è possibile fare cose molto più complesse di quelle che ci si può aspettare

-

*Scolz F.*